# Save and Restore

The [tf.train.Saver](https://tensorflow.google.cn/api_docs/python/tf/train/Saver) class provides methods to save and restore models. The
[tf.saved_model.simple_save](https://tensorflow.google.cn/api_docs/python/tf/saved_model/simple_save) function is an easy way to build a
[saved model](https://tensorflow.google.cn/api_docs/python/tf/saved_model) suitable for serving.
[Estimators](@{$guide/estimators}) automatically save and restore
variables in the `model_dir`.

## Save and restore variables

TensorFlow [Variables](https://tensorflow.google.cn/programmers_guide/variables) are the best way to represent shared, persistent state
manipulated by your program. The `tf.train.Saver` constructor adds `save` and
`restore` ops to the graph for all, or a specified list, of the variables in the
graph.  The `Saver` object provides methods to run these ops, specifying paths
for the checkpoint files to write to or read from.

`Saver` restores all variables already defined in your model. If you're
loading a model without knowing how to build its graph (for example, if you're
writing a generic program to load models), then read the
[Overview of saving and restoring models](#models) section
later in this document.

TensorFlow saves variables in binary *checkpoint files* that map variable
names to tensor values.

> Caution: TensorFlow model files are code. Be careful with untrusted code.
See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)
for details.

### Save variables

Create a `Saver` with `tf.train.Saver()` to manage all variables in the
model. For example, the following snippet demonstrates how to call the
`tf.train.Saver.save` method to save variables to checkpoint files:

In [1]:
import tensorflow as tf

# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model.
    inc_v1.op.run()
    dec_v2.op.run()
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)

C:\Users\Administrator\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Model saved in path: /tmp/model.ckpt


### Restore variables

The [`tf.train.Saver`](https://tensorflow.google.cn/api_docs/python/tf/train/Saver) object not only saves variables to checkpoint files, it
also restores variables. Note that when you restore variables you do not have
to initialize them beforehand. For example, the following snippet demonstrates
how to call the [`tf.train.Saver.restore`](https://tensorflow.google.cn/api_docs/python/tf/train/Saver#restore) method to restore variables from the
checkpoint files:

In [2]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    # Check the values of the variables
    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Model restored.
v1 : [1. 1. 1.]
v2 : [-1. -1. -1. -1. -1.]


> Note: There is not a physical file called `/tmp/model.ckpt`. It is the *prefix* of
filenames created for the checkpoint. Users only interact with the prefix
instead of physical checkpoint files.

### Choose variables to save and restore

If you do not pass any arguments to `tf.train.Saver()`, the saver handles all
variables in the graph.  Each variable is saved under the name that was passed
when the variable was created.

It is sometimes useful to explicitly specify names for variables in the
checkpoint files.  For example, you may have trained a model with a variable
named `"weights"` whose value you want to restore into a variable named
`"params"`.

It is also sometimes useful to only save or restore a subset of the variables
used by a model.  For example, you may have trained a neural net with five
layers, and you now want to train a new model with six layers that reuses the
existing weights of the five trained layers. You can use the saver to restore
the weights of just the first five layers.

You can easily specify the names and variables to save or load by passing to the
`tf.train.Saver()` constructor either of the following:

* A list of variables (which will be stored under their own names).
* A Python dictionary in which keys are the names to use and the values are the
variables to manage.

Continuing from the save/restore examples shown earlier:

In [3]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)

# Add ops to save and restore only `v2` using the name "v2"
saver = tf.train.Saver({"v2": v2})

# Use the saver object normally after that.
with tf.Session() as sess:
    # Initialize v1 since the saver will not.
    v1.initializer.run()
    saver.restore(sess, "/tmp/model.ckpt")

    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
v1 : [0. 0. 0.]
v2 : [-1. -1. -1. -1. -1.]


Notes:

*  You can create as many `Saver` objects as you want if you need to save and
   restore different subsets of the model variables.  The same variable can be
   listed in multiple saver objects; its value is only changed when the
   `Saver.restore()` method is run.

*  If you only restore a subset of the model variables at the start of a
   session, you have to run an initialize op for the other variables.  See
   [tf.variables_initializer](https://tensorflow.google.cn/api_docs/python/tf/variables_initializer) for more information.

*  To inspect the variables in a checkpoint, you can use the
   [`inspect_checkpoint`](https://www.tensorflow.org/code/tensorflow/python/tools/inspect_checkpoint.py)
   library, particularly the `print_tensors_in_checkpoint_file` function.

*  By default, `Saver` uses the value of the [tf.Variable.name](https://tensorflow.google.cn/api_docs/python/tf/Variable#name) property
   for each variable.  However, when you create a `Saver` object, you may
   optionally choose names for the variables in the checkpoint files.


### Inspect variables in a checkpoint

We can quickly inspect variables in a checkpoint with the
[`inspect_checkpoint`](https://www.tensorflow.org/code/tensorflow/python/tools/inspect_checkpoint.py) library.

Continuing from the save/restore examples shown earlier:

In [6]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name="", all_tensors=True)

# tensor_name:  v1
# [ 1.  1.  1.]
# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

# print only tensor v1 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v1', all_tensors=False)

# tensor_name:  v1
# [ 1.  1.  1.]

# print only tensor v2 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v2', all_tensors=False)

# tensor_name:  v2
# [-1. -1. -1. -1. -1.] 

tensor_name:  v1
[1. 1. 1.]
tensor_name:  v2
[-1. -1. -1. -1. -1.]
tensor_name:  v1
[1. 1. 1.]
tensor_name:  v2
[-1. -1. -1. -1. -1.]


## Save and restore models

Use `SavedModel` to save and load your model—variables, the graph, and the
graph's metadata. This is a language-neutral, recoverable, hermetic
serialization format that enables higher-level systems and tools to produce,
consume, and transform TensorFlow models. TensorFlow provides several ways to
interact with `SavedModel`, including the [tf.saved_model](https://tensorflow.google.cn/api_docs/python/tf/saved_model) APIs,
[tf.estimator.Estimator](https://tensorflow.google.cn/api_docs/python/tf/estimator/Estimator), and a command-line interface.


## Build and load a SavedModel

### Simple save

The easiest way to create a `SavedModel` is to use the [tf.saved_model.simple_save](https://tensorflow.google.cn/api_docs/python/tf/saved_model/simple_save)
function:

In [ ]:
simple_save(session,
            export_dir,
            inputs={"x": x, "y": y},
            outputs={"z": z})

This configures the `SavedModel` so it can be loaded by
[TensorFlow serving](/serving/serving_basic) and supports the
[Predict API](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/apis/predict.proto).
To access the classify, regress, or multi-inference APIs, use the manual
`SavedModel` builder APIs or an [tf.estimator.Estimator](https://tensorflow.google.cn/api_docs/python/tf/estimator/Estimator).

### Manually build a SavedModel

If your use case isn't covered by [tf.saved_model.simple_save](https://tensorflow.google.cn/api_docs/python/tf/saved_model/simple_save), use the manual
[builder APIs](https://tensorflow.google.cn/api_docs/python/tf/saved_model/builder) to create a `SavedModel`.

The [tf.saved_model.builder.SavedModelBuilder](https://tensorflow.google.cn/api_docs/python/tf/saved_model/builder/SavedModelBuilder) class provides functionality to
save multiple `MetaGraphDef`s.  A **MetaGraph** is a dataflow graph, plus
its associated variables, assets, and signatures.  A **`MetaGraphDef`**
is the protocol buffer representation of a MetaGraph.  A **signature** is
the set of inputs to and outputs from a graph.

If assets need to be saved and written or copied to disk, they can be provided
when the first `MetaGraphDef` is added. If multiple `MetaGraphDef`s are
associated with an asset of the same name, only the first version is retained.

Each `MetaGraphDef` added to the SavedModel must be annotated with
user-specified tags. The tags provide a means to identify the specific
`MetaGraphDef` to load and restore, along with the shared set of variables
and assets. These tags
typically annotate a `MetaGraphDef` with its functionality (for example,
serving or training), and optionally with hardware-specific aspects (for
example, GPU).

For example, the following code suggests a typical way to use
`SavedModelBuilder` to build a SavedModel:

In [ ]:
export_dir = ...
...
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.Session(graph=tf.Graph()) as sess:
    ...
    builder.add_meta_graph_and_variables(sess,
                                       [tag_constants.TRAINING],
                                       signature_def_map=foo_signatures,
                                       assets_collection=foo_assets,
                                       strip_default_attrs=True)
...
# Add a second MetaGraphDef for inference.
with tf.Session(graph=tf.Graph()) as sess:
    ...
    builder.add_meta_graph([tag_constants.SERVING], strip_default_attrs=True)
...
builder.save()

#### Forward compatibility via `strip_default_attrs=True`

Following the guidance below gives you forward compatibility only if the set of
Ops has not changed.

The [SavedModelBuilder](https://tensorflow.google.cn/api_docs/python/tf/saved_model/builder/SavedModelBuilder) class allows
users to control whether default-valued attributes must be stripped from the
[NodeDefs](https://tensorflow.google.cn/extend/tool_developers/index#nodes)
while adding a meta graph to the SavedModel bundle. Both
[SavedModelBuilder.add_meta_graph_and_variables](https://tensorflow.google.cn/api_docs/python/tf/saved_model/builder/SavedModelBuilder#add_meta_graph_and_variables)
and [SavedModelBuilder.add_meta_graph](https://tensorflow.google.cn/api_docs/python/tf/saved_model/builder/SavedModelBuilder#add_meta_graph)
methods accept a Boolean flag `strip_default_attrs` that controls this behavior.

If `strip_default_attrs` is `False`, the exported [tf.MetaGraphDef](https://tensorflow.google.cn/api_docs/python/tf/MetaGraphDef) will have
the default valued attributes in all its [tf.NodeDef](https://tensorflow.google.cn/api_docs/python/tf/NodeDef) instances.
This can break forward compatibility with a sequence of events such as the
following:

*  An existing Op (`Foo`) is updated to include a new attribute (`T`) with a
   default (`bool`) at version 101.
*  A model producer such as a "trainer binary" picks up this change (version 101)
   to the `OpDef` and re-exports an existing model that uses Op `Foo`.
*  A model consumer (such as [Tensorflow Serving](/serving)) running an older
   binary (version 100) doesn't have attribute `T` for Op `Foo`, but tries to
   import this model. The model consumer doesn't recognize attribute `T` in a
   `NodeDef` that uses Op `Foo` and therefore fails to load the model.
*  By setting `strip_default_attrs` to True, the model producers can strip away
   any default valued attributes in the `NodeDefs`. This helps ensure that newly
   added attributes with defaults don't cause older model consumers to fail
   loading models regenerated with newer training binaries.

See [compatibility guidance](./version_compat.md)
for more information.

### Loading a SavedModel in Python

The Python version of the SavedModel
[loader](https://tensorflow.google.cn/api_docs/python/tf/saved_model/loader)
provides load and restore capability for a SavedModel. The `load` operation
requires the following information:

* The session in which to restore the graph definition and variables.
* The tags used to identify the MetaGraphDef to load.
* The location (directory) of the SavedModel.

Upon a load, the subset of variables, assets, and signatures supplied as part of
the specific MetaGraphDef will be restored into the supplied session.

In [ ]:
export_dir = ...
...
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tag_constants.TRAINING], export_dir)
    ...

### Load a SavedModel in C++

The C++ version of the SavedModel
[loader](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/cc/saved_model/loader.h)
provides an API to load a SavedModel from a path, while allowing
`SessionOptions` and `RunOptions`.
You have to specify the tags associated with the graph to be loaded.
The loaded version of SavedModel is referred to as `SavedModelBundle`
and contains the MetaGraphDef and the session within which it is loaded.

In [ ]:
const string export_dir = ...
SavedModelBundle bundle;
...
LoadSavedModel(session_options, run_options, export_dir, {kSavedModelTagTrain},
               &bundle);

### Load and serve a SavedModel in TensorFlow serving

You can easily load and serve a SavedModel with the TensorFlow Serving Model
Server binary. See [instructions](https://www.tensorflow.org/serving/setup#installing_using_apt-get)
on how to install the server, or build it if you wish.

Once you have the Model Server, run it with:

In [ ]:
tensorflow_model_server --port=port-numbers --model_name=your-model-name --model_base_path=your_model_base_path

Set the port and model_name flags to values of your choosing. The
model_base_path flag expects to be to a base directory, with each version of
your model residing in a numerically named subdirectory. If you only have a
single version of your model, simply place it in a subdirectory like so:
* Place the model in /tmp/model/0001
* Set model_base_path to /tmp/model

Store different versions of your model in numerically named subdirectories of a
common base directory. For example, suppose the base directory is `/tmp/model`.
If you have only one version of your model, store it in `/tmp/model/0001`. If
you have two versions of your model, store the second version in
`/tmp/model/0002`, and so on.  Set the `--model-base_path` flag to the base
directory (`/tmp/model`, in this example).  TensorFlow Model Server will serve
the model in the highest numbered subdirectory of that base directory.

### Standard constants

SavedModel offers the flexibility to build and load TensorFlow graphs for a
variety of use-cases. For the most common use-cases, SavedModel's APIs
provide a set of constants in Python and C++ that are easy to
reuse and share across tools consistently.

#### Standard MetaGraphDef tags

You may use sets of tags to uniquely identify a `MetaGraphDef` saved in a
SavedModel. A subset of commonly used tags is specified in:

* [Python](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/tag_constants.py)
* [C++](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/cc/saved_model/tag_constants.h)

## Using SavedModel with Estimators

After training an `Estimator` model, you may want to create a service
from that model that takes requests and returns a result.  You can run such a
service locally on your machine or deploy it in the cloud.

To prepare a trained Estimator for serving, you must export it in the standard
SavedModel format. This section explains how to:

* Specify the output nodes and the corresponding
  [APIs](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/apis/prediction_service.proto)
  that can be served (Classify, Regress, or Predict).
* Export your model to the SavedModel format.
* Serve the model from a local server and request predictions.


### Prepare serving inputs

During training, an [`input_fn()`](https://tensorflow.google.cn/get_started/premade_estimators#input_fn) ingests data
and prepares it for use by the model.  At serving time, similarly, a
`serving_input_receiver_fn()` accepts inference requests and prepares them for
the model.  This function has the following purposes:

*  To add placeholders to the graph that the serving system will feed
   with inference requests.
*  To add any additional ops needed to convert data from the input format
   into the feature `Tensor`s expected by the model.

The function returns a [tf.estimator.export.ServingInputReceiver](https://tensorflow.google.cn/api_docs/python/tf/estimator/export/ServingInputReceiver) object,
which packages the placeholders and the resulting feature `Tensor`s together.

A typical pattern is that inference requests arrive in the form of serialized
`tf.Example`s, so the `serving_input_receiver_fn()` creates a single string
placeholder to receive them.  The `serving_input_receiver_fn()` is then also
responsible for parsing the `tf.Example`s by adding a [tf.parse_example](https://tensorflow.google.cn/api_docs/python/tf/parse_example) op to
the graph.

When writing such a `serving_input_receiver_fn()`, you must pass a parsing
specification to [tf.parse_example](https://tensorflow.google.cn/api_docs/python/tf/parse_example) to tell the parser what feature names to
expect and how to map them to `Tensor`s. A parsing specification takes the
form of a dict from feature names to [tf.FixedLenFeature](https://tensorflow.google.cn/api_docs/python/tf/FixedLenFeature), [tf.VarLenFeature](https://tensorflow.google.cn/api_docs/python/tf/VarLenFeature),
and [tf.SparseFeature](https://tensorflow.google.cn/api_docs/python/tf/SparseFeature).  Note this parsing specification should not include
any label or weight columns, since those will not be available at serving
time&mdash;in contrast to a parsing specification used in the `input_fn()` at
training time.

In combination, then:

In [ ]:
feature_spec = {'foo': tf.FixedLenFeature(...),
                'bar': tf.VarLenFeature(...)}

def serving_input_receiver_fn():
    """An input receiver that expects a serialized tf.Example."""
    serialized_tf_example = tf.placeholder(dtype=tf.string,
                                         shape=[default_batch_size],
                                         name='input_example_tensor')
    receiver_tensors = {'examples': serialized_tf_example}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)